In [1]:
import pylhe
import ROOT
import math
import os
import pandas as pd
import uproot
import numpy as np
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.20/04


In [2]:
##### setting file path #####

sig_rootpath = "/data/Chen-Wang/zaSIG10/Events/run_01/tag_1_delphes_events.root"
sig_file = uproot.open(sig_rootpath)

bg_aa_rootpath = "/data/Chen-Wang/llaaBG/Events/run_01/tag_1_delphes_events.root"
bg_aa_file = uproot.open(bg_aa_rootpath)

bg_jet_rootpath = "/data/Chen-Wang/lljBG/Events/run_01/tag_1_delphes_events.root"
bg_jet_file = uproot.open(bg_jet_rootpath)

In [3]:
##### check final state electron(muon) in background file #####

num = 8
bg_aa_electron = bg_aa_file["Delphes;1"]["Electron"].array()
bg_aa_muon = bg_aa_file["Delphes;1"]["Muon"].array()
bg_aa_photon = bg_aa_file["Delphes;1"]["Event.Number"].array()
print("In the", num, "th 2 photon background event:\n")
print("number of electron: ", bg_aa_electron[num])
print("number of muon: ", bg_aa_muon[num])
print("number of photon: ", bg_aa_photon[num])

In the 8 th 2 photon background event:

number of electron:  2
number of muon:  0
number of photon:  [8]


In [4]:
##### check final state electron(muon) in signal file #####

sig_electron = bg_jet_file["Delphes;1"]["Electron"].array()
sig_muon = bg_jet_file["Delphes;1"]["Muon.Particle"].array()
sig_photon = bg_jet_file["Delphes;1"]["Photon"].array()
sig_photon_particles = bg_jet_file["Delphes;1"]["Photon.Particles"].array()
sig_jet = bg_jet_file["Delphes;1"]["Jet"].array()
sig_jet_particles = bg_jet_file["Delphes;1"]["Jet.Particles"].array()
sig_jet_theta = bg_jet_file["Delphes;1"]["Jet.EhadOverEem"].array()
sig_jet_mass = bg_jet_file["Delphes;1"]["Jet.Mass"].array()
sig_num = bg_jet_file["Delphes;1"]["Event.Number"].array()

In [5]:
num = 8250
print("In the", num, "th signal event:\n")
print("number of electron: ", sig_electron[num])
print("number of muon: ", sig_muon[num])
print("number of photon: ", sig_photon[num])
print("photon particles:", sig_photon_particles[num])
print("number of jets:", sig_jet[num])
print("jet particles:", sig_jet_particles[num])
print("jet theta:", sig_jet_theta[num])
print("jet mass:", sig_jet_mass[num])
print("event number:", sig_num[num])

In the 8250 th signal event:

number of electron:  0
number of muon:  [<TRef 471> <TRef 469>]
number of photon:  0
photon particles: []
number of jets: 2
jet particles: [[958, 773, 775, 668, 893, 671, 894, 769, 771, 772, 892, 956, 955, 891], [875, 947, 874, 948, 882, 881]]
jet theta: [0.77246284 0.        ]
jet mass: [9.29327   2.0195355]
event number: [50]


In [6]:
##### set signal event content #####

events = [bg_jet_file["Delphes;1"]["Particle.Status"].array(),
        bg_jet_file["Delphes;1"]["Particle.M1"].array(),
        bg_jet_file["Delphes;1"]["Particle.M2"].array(),
        bg_jet_file["Delphes;1"]["Particle.D1"].array(),
        bg_jet_file["Delphes;1"]["Particle.D2"].array(),
        bg_jet_file["Delphes;1"]["Particle.PID"].array(),
        bg_jet_file["Delphes;1"]["Particle.PT"].array(),
        bg_jet_file["Delphes;1"]["Particle.Eta"].array(),
        bg_jet_file["Delphes;1"]["Particle.Phi"].array(),
        bg_jet_file["Delphes;1"]["Particle.Mass"].array()
        ]

##### reshape the signal data #####

events = np.expand_dims(events, axis=-1)
events = events.transpose((1,0,2))
events = np.squeeze(events,axis=(2,))

##### set features #####

_Status, _M1, _M2, _D1, _D2, _PID, _PT, _Eta, _Phi, _Mass = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 
Labels = ["Status", "M1", "M2", "D1", "D2", "PID", "PT", "Eta", "Phi", "Mass"]

##### Show truth record #####

print("Index", "\t","Status", "\t","M1"
    "\t","M2" ,"\t","D1", "\t","D2", "\t","PID", 
    "\t\t","PT" "\t","Eta", "\t\t","Phi", 
    "\t\t","Mass")

event_num = 8250
for j in range(len(events[event_num][0])):
    print(j, "\t", events[event_num][_Status][j],"\t\t",
    events[event_num][_M1][j], "\t", events[event_num][_M2][j],
    "\t", events[event_num][_D1][j], "\t", events[event_num][_D2][j],
    "\t", str(events[event_num][_PID][j]).ljust(12, ' '), "\t", round(events[event_num][_PT][j],1),  "\t",
        str(round(events[event_num][_Eta][j],2)).ljust(12, ' ') , "\t",
        str(round(events[event_num][_Phi][j],3)).ljust(12, ' '), "\t",
        round(events[event_num][_Mass][j],3))

Index 	 Status 	 M1	 M2 	 D1 	 D2 	 PID 		 PT	 Eta 		 Phi 		 Mass
0 	 21 		 5 	 -1 	 3 	 4 	 2            	 0.0 	 999.9        	 0.0          	 0.0
1 	 43 		 5 	 -1 	 15 	 15 	 21           	 51.3 	 0.31         	 -2.193       	 0.0
2 	 21 		 6 	 -1 	 3 	 4 	 21           	 0.0 	 -999.9       	 0.0          	 0.0
3 	 22 		 0 	 2 	 8 	 8 	 23           	 95.4 	 -1.55        	 -3.006       	 90.768
4 	 23 		 0 	 2 	 9 	 9 	 2            	 95.4 	 -1.7         	 0.135        	 0.0
5 	 41 		 10 	 -1 	 0 	 1 	 2            	 0.0 	 999.9        	 -2.554       	 0.0
6 	 42 		 12 	 -1 	 2 	 2 	 21           	 0.0 	 -999.9       	 -2.159       	 0.0
7 	 43 		 12 	 -1 	 18 	 18 	 21           	 7.7 	 -3.6         	 -0.242       	 0.0
8 	 44 		 3 	 -1 	 13 	 13 	 23           	 75.8 	 -1.44        	 2.935        	 90.768
9 	 44 		 4 	 -1 	 14 	 14 	 2            	 107.4 	 -1.73        	 0.246        	 0.0
10 	 42 		 19 	 -1 	 5 	 5 	 2            	 0.0 	 999.9        	 2.93         	 0.0
11 	 43 	

In [7]:
##### show the branch in Delphes;1 #####

sig_file["Delphes;1"].show()

Event                      TStreamerInfo              asdtype('>i4')
Event.fUniqueID            TStreamerBasicType         asjagged(asdtype('>u4'))
Event.fBits                TStreamerBasicType         asjagged(asdtype('>u4'))
Event.Number               TStreamerBasicType         asjagged(asdtype('>i8'))
Event.ReadTime             TStreamerBasicType         asjagged(asdtype('>f4'))
Event.ProcTime             TStreamerBasicType         asjagged(asdtype('>f4'))
Event.ProcessID            TStreamerBasicType         asjagged(asdtype('>i4'))
Event.MPI                  TStreamerBasicType         asjagged(asdtype('>i4'))
Event.Weight               TStreamerBasicType         asjagged(asdtype('>f4'))
Event.CrossSection         TStreamerBasicType         asjagged(asdtype('>f4'))
Event.CrossSectionError    TStreamerBasicType         asjagged(asdtype('>f4'))
Event.Scale                TStreamerBasicType         asjagged(asdtype('>f4'))
Event.AlphaQED             TStreamerBasicType         asjagged

In [8]:
for i in range(100):
    print(i, sig_jet_theta[i*100+50])

0 [0.5446567]
1 []
2 [1.6137347  0.22430362]
3 [1.5814421]
4 [5.827269]
5 [9.717795  0.6777622]
6 []
7 [4.1881595]
8 [1.4575697]
9 [13.049227   1.9161378]
10 [0.13845813]
11 [1.8496426]
12 [28.784735]
13 [ 1.2243136  0.        10.190496  15.059846 ]
14 [2.2018151]
15 [5.4536796]
16 [2.2397676 3.9527538 0.       ]
17 []
18 [0.]
19 []
20 [999.9         0.        999.9         2.8385549]
21 [999.9         1.7283684]
22 [24.369827]
23 [26.555368]
24 [7.8125043]
25 [0.        1.7643114]
26 [1.6201031]
27 []
28 [51.821198]
29 [2.8864982 1.2056292]
30 []
31 [0.79136944]
32 []
33 [4.7814465]
34 [0.8821611 8.05444   6.5640993]
35 [2.5631008 2.080143 ]
36 [0.        7.6254563]
37 [0.11145186 2.288829  ]
38 [1.2371505]
39 [3.730611  1.0795512 2.8426876]
40 [  8.6104965 999.9         8.394849 ]
41 [3.443758]
42 [0.]
43 [10.976173    0.26490358  0.17489631]
44 [1.4970754]
45 [0.14935909]
46 [999.9]
47 [25.003553    0.74077624]
48 []
49 [1.6515632 0.4198598 2.0761895]
50 [0.9350072]
51 [4.4966607 0.